In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import warnings
from colorama import Fore, Style
import gc
import pickle

from sklearn.calibration import CalibrationDisplay
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = 250

In [2]:
data = pd.read_feather('data/train_data.ftr')
data.rename({'S_2': 'date'}, axis=1, inplace=True)
data.head()

,customer_ID,date,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,NaN,NaN,0.000630,0.080994,0.708984,0.170654,0.006203,0.358643,0.525391,0.255615,NaN,0.063904,0.059418,0.006466,0.148682,1.335938,0.008209,0.001423,0.207275,0.736328,0.096191,NaN,0.023376,0.002768,0.008324,1.001953,0.008301,0.161377,0.148315,0.922852,0.354492,0.151978,0.118103,0.001882,0.158569,0.065735,0.018387,0.063660,0.199585,0.308350,0.016357,0.401611,0.091064,CR,O,0.007126,0.007664,NaN,0.652832,0.008522,NaN,0.004730,6.0,0.271973,0.008362,0.515137,0.002644,0.009010,0.004807,0.008339,0.119385,0.004803,0.108276,0.050873,NaN,0.007553,0.080444,0.069092,NaN,0.004326,0.007561,NaN,0.007729,0.000272,0.001575,0.004238,0.001434,NaN,0.002272,0.004059,0.007122,0.002457,0.002310,0.003532,0.506836,0.008034,1.009766,0.084656,0.003820,0.007042,0.000438,0.006451,0.000830,0.005054,NaN,0.0,0.005722,0.007084,NaN,0.000198,0.008911,NaN,1.0,0.002537,0.005177,0.006626,0.009705,0.007782,0.002449,1.000977,0.002665,0.007481,0.006893,1.503906,1.005859,0.003569,0.008873,0.003948,0.003647,0.004951,0.894043,0.135620,0.911133,0.974609,0.001244,0.766602,1.008789,1.004883,0.893555,NaN,0.669922,0.009972,0.004570,NaN,1.008789,2.0,NaN,0.004326,NaN,NaN,NaN,1.007812,0.210083,0.676758,0.007874,1.0,0.238281,0.0,4.0,0.232178,0.236206,0.0,0.702148,0.434326,0.003057,0.686523,0.008743,1.0,1.002930,1.007812,1.000000,0.006805,NaN,0.002052,0.005970,NaN,0.004345,0.001534,NaN,NaN,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,NaN,NaN,0.002525,0.069397,0.712891,0.113220,0.006207,0.353516,0.521484,0.223389,NaN,0.065247,0.057739,0.001614,0.149780,1.339844,0.008369,0.001984,0.202759,0.720703,0.099792,NaN,0.030594,0.002748,0.002481,1.008789,0.005135,0.140991,0.143555,0.919434,0.326660,0.156250,0.118713,0.001610,0.148438,0.093933,0.013039,0.065491,0.151367,0.265137,0.017685,0.406250,0.086792,CR,O,0.002413,0.007149,NaN,0.646973,0.002237,NaN,0.003880,6.0,0.188965,0.004028,0.509277,0.004192,0.007843,0.001283,0.006523,0.140625,0.000094,0.101013,0.040466,NaN,0.004833,0.081421,0.074158,NaN,0.004204,0.005302,NaN,0.001864,0.000978,0.009895,0.007599,0.000509,NaN,0.009811,0.000126,0.005966,0.000395,0.001327,0.007774,0.500977,0.000761,1.009766,0.081848,0.000347,0.007790,0.004311,0.002333,0.009468,0.003754,NaN,0.0,0.007584,0.006676,NaN,0.001143,0.005905,NaN,1.0,0.008430,0.008980,0.001854,0.009926,0.005989,0.002247,1.006836,0.002508,0.006828,0.002836,1.503906,1.005859,0.000571,0.000391,0.008354,0.008850,0.003180,0.902344,0.136353,0.919922,0.975586,0.004562,0.786133,1.000000,1.003906,0.906738,NaN,0.668457,0.003922,0.004654,NaN,1.002930,2.0,NaN,0.008705,NaN,NaN,NaN,1.007812,0.184082,0.822266,0.003445,1.0,0.247192,0.0,4.0,0.243530,0.241943,0.0,0.707031,0.430420,0.001306,0.686523,0.000755,1.0,1.008789,1.003906,1.008789,0.004406,NaN,0.001034,0

B_31 make cat feature


In [3]:
info_columns = ['date', 'target', 'customer_ID']
bin_columns = ['D_87', 'B_31']
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
num_columns = [x for x in data.columns if x not in info_columns+bin_columns+cat_columns]

encoders = []
for feature in cat_columns:
    cat_encoder = LabelEncoder()
    data[feature] = cat_encoder.fit_transform(data[feature])
    encoders.append(cat_encoder)
with open('models/cat_encoder.pkl', 'wb') as f:
    pickle.dump(encoders, f)

In [10]:
def make_num_features(df, num_columns):
    new_data = df[num_columns+['customer_ID']].groupby('customer_ID').agg(['mean', 'median', 'std', 'min', 'max'])
    new_data.columns = [f'{x[0]}_{x[1]}' for x in new_data.columns]
    return new_data


def make_categorical_features(df):
    new_data = df.groupby('customer_ID').agg(['nunique'])
    new_data.columns = [f'{x[0]}_{x[1]}' for x in new_data.columns]
    return new_data


def make_last_features(df, num_cols, cat_cols):
    cid = df['customer_ID']
    last = (cid != np.roll(cid, -1))
    data_last = df.loc[last, num_cols+cat_cols].rename(columns={f: f"{f}_last" for f in num_cols+cat_cols}).set_index(np.asarray(cid[last]))
    return data_last, [f'{x}_last' for x in cat_cols]


cat_features = make_categorical_features(data[cat_columns + ['customer_ID']])
num_features = make_num_features(data, num_columns)
last_features, cat_cols = make_last_features(data, num_columns, cat_columns)
all_features = cat_features.merge(num_features, how='left', left_index=True, right_index=True).merge(
                                    last_features, how='left', left_index=True, right_index=True)
del cat_features, num_features, last_features

In [12]:
all_features

,B_30_nunique,B_30_count,B_38_nunique,B_38_count,D_114_nunique,D_114_count,D_116_nunique,D_116_count,D_117_nunique,D_117_count,D_120_nunique,D_120_count,D_126_nunique,D_126_count,D_63_nunique,D_63_count,D_64_nunique,D_64_count,D_66_nunique,D_66_count,D_68_nunique,D_68_count,P_2_mean,P_2_median,P_2_std,P_2_min,P_2_max,D_39_mean,D_39_median,D_39_std,D_39_min,D_39_max,B_1_mean,B_1_median,B_1_std,B_1_min,B_1_max,B_2_mean,B_2_median,B_2_std,B_2_min,B_2_max,R_1_mean,R_1_median,R_1_std,R_1_min,R_1_max,S_3_mean,S_3_median,S_3_std,S_3_min,S_3_max,D_41_mean,D_41_median,D_41_std,D_41_min,D_41_max,B_3_mean,B_3_median,B_3_std,B_3_min,B_3_max,D_42_mean,D_42_median,D_42_std,D_42_min,D_42_max,D_43_mean,D_43_median,D_43_std,D_43_min,D_43_max,D_44_mean,D_44_median,D_44_std,D_44_min,D_44_max,B_4_mean,B_4_median,B_4_std,B_4_min,B_4_max,D_45_mean,D_45_median,D_45_std,D_45_min,D_45_max,B_5_mean,B_5_median,B_5_std,B_5_min,B_5_max,R_2_mean,R_2_median,R_2_std,R_2_min,R_2_max,D_46_mean,D_46_median,D_46_std,D_46_min,D_46_max,D_47_mean,D_47_median,D_47_std,D_47_min,D_47_max,D_48_mean,D_48_median,D_48_std,D_48_min,D_48_max,D_49_mean,D_49_median,D_49_std,D_49_min,D_49_max,B_6_mean,B_6_median,B_6_std,B_6_min,B_6_max,B_7_mean,B_7_median,B_7_std,...,D_69_last,B_22_last,D_70_last,D_71_last,D_72_last,S_15_last,B_23_last,D_73_last,P_4_last,D_74_last,D_75_last,D_76_last,B_24_last,R_7_last,D_77_last,B_25_last,B_26_last,D_78_last,D_79_last,R_8_last,R_9_last,S_16_last,D_80_last,R_10_last,R_11_last,B_27_last,D_81_last,D_82_last,S_17_last,R_12_last,B_28_last,R_13_last,D_83_last,R_14_last,R_15_last,D_84_last,R_16_last,B_29_last,S_18_last,D_86_last,R_17_last,R_18_last,D_88_last,S_19_last,R_19_last,B_32_last,S_20_last,R_20_last,R_21_last,B_33_last,D_89_last,R_22_last,R_23_last,D_91_last,D_92_last,D_93_last,D_94_last,R_24_last,R_25_last,D_96_last,S_22_last,S_23_last,S_24_last,S_25_last,S_26_last,D_102_last,D_103_last,D_104_last,D_105_last,D_106_last,D_107_last,B_36_last,B_37_last,R_26_last,R_27_last,D_108_last,D_109_last,D_110_last,D_111_last,B_39_last,D_112_last,B_40_last,S_27_last,D_113_last,D_115_last,D_118_last,D_119_last,D_121_last,D_122_last,D_123_last,D_124_last,D_125_last,D_127_last,D_128_last,D_129_last,B_41_last,B_42_last,D_130_last,D_131_last,D_132_last,D_133_last,R_28_last,D_134_last,D_135_last,D_136_last,D_137_last,D_138_last,D_139_last,D_140_last,D_141_last,D_142_last,D_143_last,D_144_last,D_145_last,B_30_last,B_38_last,D_114_last,D_116_last,D_117_last,D_120_last,D_126_last,D_63_last,D_64_last,D_66_last,D_68_last
customer_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,13,1,13,1,13,1,13,1,13,1,13,1,13,1,13,1,13,1,13,1,13,0.933594,0.938477,0.024194,0.868652,0.960449,0.010704,0.002483,0.024440,0.001082,0.091492,0.012009,0.013687,0.006546,0.001930,0.021652,1.004883,1.004883,0.003248,1.000000,1.009766,0.004509,0.004509,0.003081,0.000263,0.009224,0.113220,0.110962,0.011665,0.098877,0.135010,0.005020,0.004654,0.003331,0.000685,0.009857,0.006454,0.007175,0.002942,0.000783,0.009865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004673,0.004128,0.002823,0.000630,0.009399,0.040314,0.038849,0.031726,0.000836,0.080994,0.725586,0.723145,0.009530,0.708984,0.740234,0.146606,0.143677,0.047200,0.060486,0.231689,0.005753,0.006207,0.002928,0.001436,0.009918,0.377930,0.358643,0.085663,0.230957,0.519531,0.532715,0.533203,0.006533,0.521484,0.541992,0.240967,0.230225,0.076889,0.135620,0.403564,NaN,NaN,NaN,NaN,NaN,0.113525,0.104492,0.047368,0.063904,0.221924,0.036621,0.048920,0.023195,...,0.005470,0.008049,0.008255,0.377930,0.006969,0.304688,0.040375,NaN,0.006565,0.001299,0.001352,NaN,0.002230,0.009163,0.421387,0.006474,0.001067,0.000747,0.002193,0.006344,NaN,0.000026,0.208252,0.001058,0.009354,0.007236,0.009178,0.5073